In [1]:
def read_In(fileName):
    with open(fileName, 'r') as f:
        composition = f.readlines()
    n = len(composition)
    for i in range(n):
        if composition[i][-1] == '\n':
            composition[i] = composition[i][:-1]
    composition.sort()
    return composition

In [2]:
def split_Graph(graph):
    n = len(graph)
    prefix = [0] * n
    surffix = [0] * n
    for i in range(n):
        prefix[i], surffix[i] = graph[i].strip().split('|')
    return prefix, surffix

In [3]:
def graph_to_dictionary(prefix, surffix):
    n = len(prefix)
    dictionary = {}
    for i in range(n):
        if prefix[i] not in dictionary.keys():
            dictionary[(prefix[i][:-1], surffix[i][:-1])] = (prefix[i][1:], surffix[i][1:])
        else:
            dictionary[(prefix[i][:-1], surffix[i][:-1])].append((prefix[i][1:], surffix[i][1:]))
    return dictionary

In [4]:
def DeBruijnGraph(composition, k):
    prefix_dictionary = {}
    # n = len(composition)
    
    for keys in composition.keys():
        if keys not in prefix_dictionary.keys():
            prefix_dictionary[keys] = [composition[keys]]
        else:
            prefix_dictionary[keys] += [composition[keys]]
    return prefix_dictionary

In [5]:
def remove_edge(dictionary, from_node, to_node):
    
    dictionary[from_node].remove(to_node)
    if not dictionary[from_node]:
        del dictionary[from_node]
    return dictionary

In [6]:
from random import choice
def find_New_Cycle(dictionary):
    
    # Randomly choose a starting point
    start_node, edges = choice(list(dictionary.items()))
    target_node = choice(edges)
    dictionary = remove_edge(dictionary, start_node, target_node)
    
    cycle = [start_node, target_node]

    current_node = target_node
    
    while current_node != start_node:
        edges = dictionary[current_node]
        # extract one element in edges (if more than one nodes inside)
        target_node = choice(edges)
        dictionary = remove_edge(dictionary, current_node, target_node)
        current_node = target_node
        cycle.append(current_node)
        # print(cycle)
    return cycle

In [7]:
def find_Eulerian_Cycle(dictionary):

    cycle = find_New_Cycle(dictionary)
    
    # while dictionary is not empty, do the while loop
    while dictionary:
        # search for node with unused egdes in dictionary
        potential_starts = [(index, node) for index, node in enumerate(cycle) if node in dictionary]
        
        index, new_start = choice(potential_starts)
        print(new_start)
        # form new cycle:
        # start at new_start node chosen above and then randomly walking
        new_cycle = cycle[index:] + cycle[1 : index + 1]   # change start node of the existing cycle

        target_node = choice(dictionary[new_start])
        dictionary = remove_edge(dictionary, new_start, target_node)
        current_node = target_node
        new_cycle.append(current_node)
        
        while current_node != new_start:
            edges = dictionary[current_node]
            target_node = choice(edges)
            dictionary = remove_edge(dictionary, current_node, target_node)
            current_node = target_node
            new_cycle.append(current_node)
        cycle = new_cycle
    # cycle = cycle[:(- k + 1)]
    return cycle

In [8]:
def find_Eulerian_Path(dictionary):
    
    # Define a dictionary to discern which nodes have unbalanced degrees
    degree_difference = {}
    for initial_node, end_nodes in dictionary.items():
        if initial_node in degree_difference:
            degree_difference[initial_node] += len(end_nodes)
        else:
            degree_difference[initial_node] = len(end_nodes)
        for end_node in end_nodes:
            if tuple(end_node) in degree_difference:
                degree_difference[tuple(end_node)] -= 1
            else:
                degree_difference[tuple(end_node)] = -1
    # print(degree_difference)                
    node_wo_outdegree = [node for node, difference in degree_difference.items() if difference == -1][0]
    node_wo_indegree = [node for node, difference in degree_difference.items() if difference == 1][0]
    # print('node wo outdegree:', node_wo_outdegree)
    # print('node wo indegree: ', node_wo_indegree)
    if node_wo_outdegree in dictionary:
        dictionary[node_wo_outdegree].append([node_wo_indegree])
    else:
        dictionary[node_wo_outdegree] = [node_wo_indegree]
        
    cycle = find_Eulerian_Cycle(dictionary)
    cc = cycle
    index = 0
    
    # Modeify the cycle generated and reshape it as:
    # node_without_indegree -> cycle -> node_without_ooutdegree
    while True:
        if cycle[index] == node_wo_outdegree and cycle[index + 1] == node_wo_indegree:
            break
        index += 1
    return cycle[index + 1:] + cycle[1:index + 1]

In [16]:
def cycle_to_string(eulerian_cycle, k, d):
    n = len(eulerian_cycle)
    string_1 = []
    string_2 = []
    for i in range(k - 1 - 1):
        string_1.append(eulerian_cycle[0][0][i])
        string_2.append(eulerian_cycle[0][1][i])
    for j in range(n):
        string_1.append(eulerian_cycle[j][0][-1])
        string_2.append(eulerian_cycle[j][1][-1])
    # print('string_1: ', string_1)
    # print('string_2: ', string_2)
    n_s = len(string_2)
    string = string_1 + string_2[n_s - k - d:]
    string = ''.join(string)
    return string

In [19]:
if __name__ == '__main__':
    k = 30
    d = 100
    graph = read_In('rtext.txt')
    prefix, surffix = split_Graph(graph)
    composition = graph_to_dictionary(prefix, surffix)
    DeBruijn = DeBruijnGraph(composition, k)
    eulerian_cycle = find_Eulerian_Path(DeBruijn)
    string = cycle_to_string(eulerian_cycle, k, d)
    print(string)

AGTGAATGACGTGGAAGGATCTCCTTCATGGGAGATTCTGGACCCTAAGCTCTGGGAAACTTGTGCCAGTGTTCACGGCTTTCGGTAGGGAACCGGATCGACGTACGCTCTAGAGTTGGCCTTGGCAGATCCCGTGCAGTTAACACAATATACGGTCGTAAGAAGGCTAGGTCTGGAAGGGCCGGTAGATCGACGTACGCTCTAGAGTTGGCCTTGGAGCGATTTCGACAGGAGCTCTTGGCATATGCGGCCAAGGGTTCGATCGACGTACGCTCGATCGACGTACGCTCTAGAGTTGGCCTTGGTAGAGTTGGCCTTGGAATCTAGGCCACTACGAGCGGATCGACGTACGCTCTAGAGTTGGCCTTGGGCACACTTTGAGACAGATTGGAGTATCCCGATGATAGCTGTGCCGAGGAAACCCAGCGAGATGTAGCAGAGCAGAATAGTCAGCTTTATCAATATCAGACGACCATGTCATTGTATATCACCGATCGACGTACGCTCTAGAGTGATCGACGTACGCTCTAGAGTTGGCCTTGGTGGCCTTGGCCTGGCCCTTTCGGCCTGGCCACATTTGCTGGATCGTTAGAGGCGTTCTGACTGCGATTATTTGTATAGTCGAGAGCGATCGCCCCCCCACGAGAGTTCGCGGACGTCATAAGCCAGGGCTTAAAAGCTACTGGTTGGAGACGGTATAGATCGGATCGACGTACGCTCTAGAGTTGGCCTTGGACGTACGCTCTAGAGTTGGCCTTGGGGACTGATAAAGTCGATCGACGTACGCTCTAGAGTTGGCCTTGGATTGAGATCGACGTACGCTCTAGAGTTGGCCTTGGCGCCTTATAAATATACCTTACGACCCCGCATTCCCACAAACTTCGTGCGGTGGCATCACGCATGAGGGTTACCAGATCGACGTACGCTCTAGAGTTGGCCTTGGAACTCCTGACCCCAAACAGGCTTATAATACTGGGACGATACTGACATAACTTCTCTG